In [31]:
import pandas as pd
import numpy as np
import yfinance as yf
import ta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError

def data_prep(start_date, end_date):
    df = yf.download('AMZN',start_date,end_date)
    APPL = yf.download('AAPL',start_date,end_date)
    MSFT = yf.download('MSFT',start_date,end_date)
    GOOG = yf.download('GOOG',start_date,end_date)
    VIX = yf.download("^VIX", start_date,end_date)
    usdx = yf.download('DX-Y.NYB', start_date,end_date)


    df = df.reset_index()
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    df['APPL'] = APPL['Close']
    df['MSFT'] = MSFT['Close']
    df['GOOG'] = GOOG['Close']
    df['VIX'] = VIX['Close']
    df['USDX'] = usdx['Close']

    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()

    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()

    macd = ema12 - ema26
    signal = macd.ewm(span=9, adjust=False).mean()
    histogram = macd - signal

    df['EMA12'] = ema12
    df['EMA26'] = ema26
    df['MACD'] = macd
    df['MACD_signal'] = signal
    df['MACD_hist'] = histogram

    print(df.info)
    df.columns = df.columns.str.replace(' ','_')
    df = df.dropna()
    df.drop(['Open', 'High', 'Low', 'Adj_Close'], axis=1, inplace=True)
    y_data = df['Close']
    df.drop(['Close'], axis=1, inplace=True)
    X_data = df
    return X_data,y_data

start_date = '2022-04-01'
end_date = '2023-03-31'
X, y = data_prep(start_date, end_date)

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

if X.shape[0] > 0 and y.shape[0] > 0:
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    y_scaler = MinMaxScaler()
    y = y_scaler.fit_transform(y.values.reshape(-1, 1))
else:
    print("Insufficient data for MinMaxScaler")






[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
<bound method DataFrame.info of                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-04-01  164.149506  165.826996  162.319504  163.559998  163.559998   
2022-04-04  164.125000  168.394501  163.205505  168.346497  168.346497   
2022-04-05  167.741501  168.110504  163.266006  164.054993  164.054993   
2022-04-06  161.650497  162.199997  157.254501  158.755997  158.755997   
2022-04-07  158.399994  160.078995  154.511505  157.784500  157.784500   
...                ...         ...  

In [32]:
train_size = int(len(X) * 0.8)
train_X, test_X = X[:train_size], X[train_size:]
train_Y, test_Y = y[:train_size], y[train_size:]

train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

model = Sequential()
model.add(LSTM(128, activation='tanh', input_shape=(train_X.shape[1], train_X.shape[2
]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, activation='tanh', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))
model.compile(loss=MeanSquaredError(), optimizer=Adam())


In [33]:
early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=1)
history = model.fit(train_X, train_Y, epochs=100, batch_size=16, validation_data=(test_X, test_Y), callbacks=[early_stop], verbose=2, shuffle=False)

test_loss = model.evaluate(test_X, test_Y, verbose=0)
yhat = model.predict(test_X)

yhat = y_scaler.inverse_transform(yhat)
test_Y = y_scaler.inverse_transform(test_Y)

print('Test loss: ', test_loss)

mae = mean_absolute_error(test_Y, yhat)
rmse = np.sqrt(mean_squared_error(test_Y, yhat))
r2 = r2_score(test_Y, yhat)

print('MAE: ', mae)
print('RMSE: ', rmse)
print('R²: ', r2)

Epoch 1/100
12/12 - 5s - loss: 0.2348 - val_loss: 0.0131 - 5s/epoch - 445ms/step
Epoch 2/100
12/12 - 0s - loss: 0.1155 - val_loss: 0.0151 - 82ms/epoch - 7ms/step
Epoch 3/100
12/12 - 0s - loss: 0.0468 - val_loss: 0.0809 - 84ms/epoch - 7ms/step
Epoch 4/100
12/12 - 0s - loss: 0.0278 - val_loss: 0.0359 - 79ms/epoch - 7ms/step
Epoch 5/100
12/12 - 0s - loss: 0.0281 - val_loss: 0.0299 - 79ms/epoch - 7ms/step
Epoch 6/100
12/12 - 0s - loss: 0.0244 - val_loss: 0.0389 - 78ms/epoch - 6ms/step
Epoch 7/100
12/12 - 0s - loss: 0.0181 - val_loss: 0.0288 - 81ms/epoch - 7ms/step
Epoch 8/100
12/12 - 0s - loss: 0.0144 - val_loss: 0.0264 - 80ms/epoch - 7ms/step
Epoch 9/100
12/12 - 0s - loss: 0.0131 - val_loss: 0.0252 - 82ms/epoch - 7ms/step
Epoch 10/100
12/12 - 0s - loss: 0.0099 - val_loss: 0.0153 - 78ms/epoch - 7ms/step
Epoch 11/100
12/12 - 0s - loss: 0.0103 - val_loss: 0.0206 - 84ms/epoch - 7ms/step
Epoch 12/100
12/12 - 0s - loss: 0.0086 - val_loss: 0.0100 - 80ms/epoch - 7ms/step
Epoch 13/100
12/12 - 0s -

In [34]:
n_future_days = 10
predictions = []

for i in range(n_future_days):
    input_data = test_X[-1].reshape((1, test_X.shape[1], test_X.shape[2]))
    next_day_predicted_price_scaled = model.predict(input_data)
    next_day_predicted_price = y_scaler.inverse_transform(next_day_predicted_price_scaled)
    predictions.append(next_day_predicted_price[0][0])

    input_data[0, 0, :-1] = next_day_predicted_price_scaled[0, 0]
    test_X = np.concatenate((test_X, input_data))
print(f"Predicted closing prices for the next {n_future_days} days: {predictions}")

1/1 [==============================] - 0s 31ms/step
Predicted closing prices for the next 10 days: [103.92395, 104.052925, 104.216415, 104.42405, 104.688324, 105.02567, 105.45788, 106.014206, 106.734535, 107.67432]
